# Project

In [ ]:
pip install implicit==0.4.4

In [1]:
!pip install lightfm

  Using cached lightfm-1.16.tar.gz (310 kB)
  Running setup.py clean for lightfm
Failed to build lightfm

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\RH\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\RH\\AppData\\Local\\Temp\\pip-install-n7i08249\\lightfm_3480cd4f79ae4722a5a508bbb474908d\\setup.py'"'"'; __file__='"'"'C:\\Users\\RH\\AppData\\Local\\Temp\\pip-install-n7i08249\\lightfm_3480cd4f79ae4722a5a508bbb474908d\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\RH\AppData\Local\Temp\pip-wheel-_w77e8qn'
       cwd: C:\Users\RH\AppData\Local\Temp\pip-install-n7i08249\lightfm_3480cd4f79ae4722a5a508bbb474908d\
  Complete output (25 lines):
  Compiling without OpenMP support.
  C:\Users\RH\Anaconda3\lib\site-packages\setuptools\dist.py:717: UserWarning: Usage of dash-


    Running setup.py install for lightfm: started
    Running setup.py install for lightfm: finished with status 'error'


In [2]:
from lightfm import LightFM

ModuleNotFoundError: No module named 'lightfm'

In [3]:
import implicit
print(implicit.__version__)

ModuleNotFoundError: No module named 'implicit'

In [2]:
import os
import pathlib
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import implicit

from scipy.sparse import csr_matrix
from implicit import als
from lightgbm import LGBMClassifier

from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

In [3]:
implicit.__version__

'0.5.2'

In [4]:
path = os.getcwd()
item_features = pd.read_csv('./data/product.zip', compression='zip',sep = ',', encoding = 'utf8')
user_features = pd.read_csv('./data/hh_demographic.zip', compression='zip',sep = ',', encoding = 'utf8')
df_train = pd.read_csv('./data/retail_train.zip', compression='zip', sep = ',', encoding = 'utf8')
df_test = pd.read_csv('./data/retail_test1.zip', compression='zip', sep = ',', encoding = 'utf8')



In [ ]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()


def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()


In [5]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'
VAL_MATCHER_WEEKS = 6

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)



df_train = prefilter_items(df_train, item_features=item_features, take_n_popular=5000)
#data[ITEM_COL] = data[ITEM_COL].astype(int)

# берем данные для тренировки matching модели
data_train_matcher = df_train[df_train['week_no'] < (df_train['week_no'].max() - VAL_MATCHER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = df_train[df_train['week_no'] >= (df_train['week_no'].max() - VAL_MATCHER_WEEKS)]

# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = df_test.copy()


# make cold-start to warm-start

#data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

# Теперь warm-start по пользователям


C:\Users\RH\Documents\GeekBrains\DataScience course\Рекомендательные системы\Project\utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


In [6]:
%%time

recommender = MainRecommender(data_train_matcher)

N_PREDICT = 50 

TOPK_RECALL = 50
TOPK_PRECISION = 5

result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]

C:\Users\RH\anaconda3\lib\site-packages\implicit\utils.py:31: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2496 [00:00<?, ?it/s]

Wall time: 15 s


In [8]:
result_eval_matcher.head(5)

,user_id,actual
0,1,"[999999, 879280, 940947, 968164, 979792, 10998..."
1,2,"[999999, 828106, 899624, 916122, 1030981, 1060..."
2,3,[999999]
3,4,"[831063, 883932, 891423, 908283, 954966, 10213..."
4,6,"[850102, 999999, 12731466, 938622, 940278, 109..."


In [ ]:
%%time
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))


In [2]:
result_eval_matcher.head(5)

NameError: name 'result_eval_matcher' is not defined